In [5]:
from Bio import SeqIO

def find_type(gb_file_name):
    types = []
    for rec in SeqIO.parse(gb_file_name, 'gb'):
#         print(rec.features)
        for feature in rec.features:
            if feature.type == "cand_cluster":
                types = feature.qualifiers['product']
#                 print(types) 
    return types
                                
test=find_type(r'C:\Users\ASUS\Desktop\ny_bigscape_in\MIBiG_ny_database\BGC0001332.gbk')
# test2=find_cat_and_type(r'C:\Users\ASUS\Desktop\ny_bigscape_in\blastp\WP_099279319.1.region001.gbk')
test3=find_type(r'C:\Users\ASUS\Desktop\ny_bigscape_in\blastp\WP_093723927.1.region001.gbk')
# test4 = find_type(r'C:\Users\ASUS\Desktop\ny_bigscape_in\blastp\WP_101419868.1.region001.gbk')
print(test)

[]


In [7]:
# FInd CAT domæne
from Bio import SeqIO
def find_cat_and_type(gb_file_name):
    BGC = []
    CATs = []
    KRDH = 0
    monomer = []
    types = []
    typ = False
    ser = False
    CAT_and_KRDH = False
    for rec in SeqIO.parse(gb_file_name, 'gb'):
        for feature in rec.features:
            if feature.type == "aSModule":
                BGC.append(feature.qualifiers['domains'])
                if 'monomer_pairings' in feature.qualifiers.keys():
                    monomer.append(feature.qualifiers['monomer_pairings'])
                else:
                    monomer.append('NaN')
                types.append(''.join(feature.qualifiers['type']))   
    for module in range(len(BGC)):
#         print(module)
        for domain in range(len(BGC[module])):
            if 'DH' in BGC[module][domain] and 'KR' in BGC[module][domain-1]:
                KRDH = int(module)
            if 'PCP' in BGC[module][domain]:
#                 print(len(BGC[module]))
                if 'AMP' in BGC[module][domain-1] and 'Condensation' in BGC[module][domain-2]:
                    CATs.append(int(module))
                elif len(BGC[module]) >3:
                    if 'AMP' in BGC[module][domain-2] and 'Condensation' in BGC[module][domain-3]:
                        CATs.append(int(module))

    CATs=sorted(CATs, reverse=True)
    for CAT in CATs:
        if KRDH-CAT <=2 and KRDH-CAT > 0:
            CAT_and_KRDH = 'before KR DH'
            if 'ser' in ''.join(monomer[CAT]):
                ser = 'serine'
        elif KRDH-CAT == -1:
            CAT_and_KRDH = 'after KR DH'

    if 'nrps' in types and 'pks' in types:
        typ = 'nrps/pks'
    elif 'nrps' in types and 'pks' not in types:
        typ = 'nrps'
    elif 'pks' in types and 'nrps' not in types:
        typ = 'pks' 
                        
#                         print(i)
    return typ, CAT_and_KRDH,ser
#                 types.append(''.join(feature.qualifiers['type']))
test=find_cat_and_type(r'C:\Users\ASUS\Desktop\ny_bigscape_in\MIBiG_ny_database\BGC0001332.gbk')
# test2=find_cat_and_type(r'C:\Users\ASUS\Desktop\ny_bigscape_in\blastp\WP_099279319.1.region001.gbk')
# test3=find_cat_and_type(r'C:\Users\ASUS\Desktop\ny_bigscape_in\blastp\WP_093723927.1.region001.gbk')
# test4 = find_cat_and_type(r'C:\Users\ASUS\Desktop\ny_bigscape_in\blastp\WP_101419868.1.region001.gbk')
print(test)

('nrps/pks', 'after KR DH', False)


In [9]:
def make_data_lst_of_lst(directory):
    lst_of_lst=[]
    import os
    for filename in os.listdir(directory):
        name=str(filename[:-4])
        gb_file_name=r'C:\Users\ASUS\Desktop\ny_bigscape_in\MIBiG_ny_database\{}'.format(filename)
        typ,CAT_and_KRDH,ser=find_cat_and_type(gb_file_name)
        lst_of_lst.append([name,typ,CAT_and_KRDH,ser])
    return lst_of_lst

tester=make_data_lst_of_lst(r'C:\Users\ASUS\Desktop\ny_bigscape_in\MIBiG_ny_database')

In [4]:
# print(tester)

In [10]:
import pandas as pd
df=pd.DataFrame(tester, columns=['name','BGC type', 'CAT->KR DH', 'ser'])

In [11]:
print(df)

            name  BGC type CAT->KR DH    ser
0     BGC0000001       pks      False  False
1     BGC0000002       pks      False  False
2     BGC0000003       pks      False  False
3     BGC0000004       pks      False  False
4     BGC0000005     False      False  False
...          ...       ...        ...    ...
1905  BGC0002032       pks      False  False
1906  BGC0002033       pks      False  False
1907  BGC0002034      nrps      False  False
1908  BGC0002035  nrps/pks      False  False
1909  BGC0002036     False      False  False

[1910 rows x 4 columns]


In [13]:
df.to_csv('BGC_type_and_CAT_mibig_new.txt', sep='\t')